# Create Dolphin InSAR Processing Environment

## 0. Import needed Python packages and write the work_dir context manager

In [1]:
import contextlib
import os
from pathlib import Path
import sys
from typing import Union
import yaml

In [2]:
@contextlib.contextmanager
def work_dir(work_pth: Union[Path, str]):
    """
    Temporarily change directories, within the scope of a with statement.
    Useful when invoking scripts that only input files from the current working directory.

    Usage:
    with work_dir(work_pth):
        do_things()
    """
    cwd = Path.cwd()
    os.chdir(work_pth)
    try:
        yield
    finally:
        os.chdir(cwd)

## 1. Clone the dolphin repository

In [3]:
dolphin_dir = Path.home()/"dolphin"
if not dolphin_dir.is_dir():
    with work_dir(dolphin_dir.parent):
        !git clone https://github.com/isce-framework/dolphin.git --branch v0.7.0

## 2. Update the dolphin conda-env.yml

- remove gdal (installed in base by other means)
- add ipython=8.16 (needed when running Jupyter Lab ~3.0)
- add jupyterlab=4.0.9 (needed to populate config used to update environment display name)
- add kernda=0.3.0 (needed to update the environment display name)
- add asf_search=6.7.2 (needed to search and download OPERA CSLC-S1 products)
- add ipyfilechooser=0.6.0 (provides a useful GUI file chooser)

In [4]:
conda_yaml_pth = list(dolphin_dir.glob("conda-env.yml"))[0]
with open(conda_yaml_pth, 'r') as f:
    try:
        env = yaml.safe_load(f)
    except yaml.YAMLError:
        raise

In [5]:
for dep in env['dependencies']:
    if 'gdal' in dep:
        env['dependencies'].remove(dep)
for pkg in [
    'ipython=8.16', 
    'ipywidgets<8.0.0',
    'jupyterlab=4.0.9', 
    'kernda=0.3.0', 
    'asf_search=6.7.2', 
    'ipyfilechooser=0.6.0'
]:
    if pkg not in env['dependencies']:
        env['dependencies'].append(pkg)

In [6]:
with open(conda_yaml_pth, 'w') as f:
    yaml.dump(env, f)

## 3. Create the `dolphin-env` conda environment and set its display name

If we don't set the display name to the environment name, it will appear in Jupyter as `Python 3`

**Option 1: in OpenSARLab**

In [7]:
# Creates the environment
prefix = Path.home()/f".local/envs/{env['name']}"
!mamba env create -f {conda_yaml_pth} --prefix {prefix} -q --force

# Without running this, every environment would appear in the GUI with the same name, 'Python 3'
!mamba run -n {env['name']} kernda --display-name {env['name']} -o {prefix}/share/jupyter/kernels/python3/kernel.json

^C
Error: kernel spec /Users/alexlewandowski/.local/envs/dolphin-env/share/jupyter/kernels/python3/kernel.json not found

ERROR conda.cli.main_run:execute(47): `conda run kernda --display-name dolphin-env -o /Users/alexlewandowski/.local/envs/dolphin-env/share/jupyter/kernels/python3/kernel.json` failed. (See above for error)


In [12]:
conda_yaml_pth

PosixPath('/Users/alexlewandowski/dolphin/conda-env.yml')

**Option 2: on local computer**

In [13]:
# Creates the environment
!mamba env create -f {conda_yaml_pth} --force

!python -m ipykernel install --user --name {env['name']}
# conda_envs_pth = Path(sys.executable).parents[2]
# Without running this, every environment would appear in the GUI with the same name, 'Python 3'
# !mamba run -n {env['name']} kernda --display-name {env['name']} -o {conda_envs_pth}/{env['name']}/share/jupyter/kernels/python3/kernel.json

conda-forge/osx-64                                          Using cache
conda-forge/noarch                                          Using cache
[+] 0.0s
[+] 0.1s
pkgs/main/osx-64 ━━━━━━━━━━╸━━━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/main/noarch ━━━━━━━━╸━━━━━━━━━━━━━━━╸━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/r/osx-64    ━━╸━━━━━━━━━━━━━━━╸━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/r/noarch    ━━━━━━━╸━━━━━━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s[+] 0.2s
pkgs/main/osx-64 ━━━━━━━━━━━━━╸━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.2s
pkgs/main/noarch ━━━━━━━━━━━╸━━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.2s
pkgs/r/osx-64    ━━━━━╸━━━━━━━━━━━━━━━╸━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.2s
pkgs/r/noarch    ━━━━━━━━━━╸━━━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.2spkgs/main/osx-64                                              No change
[+] 0.3s
pkgs/main/noarch ━━━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s Downloaded  0.3s
pkgs/r/osx-64    ━╸━━

## 4. Install dolphin in the `dolphin-env` environment with pip

In [11]:
with work_dir(dolphin_dir):
    !mamba run -n {env['name']} python -m pip install .

Processing /Users/alexlewandowski/dolphin
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for dolphin: filename=dolphin-0.7.0.post1.dev0+g3601816.d20231217-py3-none-any.whl size=140574 sha256=7525a2583c28b6f1f589aaccd7ab191afcf3f2ce6a3b5797685fbecf4a1e409e
  Stored in directory: /private/var/folders/8y/y_dbsnjn2hz68ph0zypzy4xh0000gq/T/pip-ephem-wheel-cache-zpe9oeix/wheels/77/f9/3f/59e118205edc3c8c6497449c7bfbeda5b25f3a67d6ada9ac3a
Successfully built dolphin

